In [1]:
import numpy as np

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf


In [4]:
path_to_file = "shakespeare.txt"


In [5]:
text = open(path_to_file,'r').read()


In [6]:
vocab = sorted(set(text))

In [7]:
char_to_ind = {char : ind for ind,char in enumerate(vocab)}

In [8]:
ind_to_char = np.array(vocab)

In [9]:
ind_to_char[33]

np.str_('H')

In [10]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [11]:
sample = text[:500]

In [12]:
new_sample = encoded_text[:500]

In [13]:
new_sample

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

In [14]:
seq_len = 120

In [15]:
total_num_seq = len(text) // (seq_len+1)

In [16]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [17]:
char_dataset.batch

<bound method DatasetV2.batch of <_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>>

In [18]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [19]:
def create_seq_targets(seq):
    input_txt = seq[:-1] #Hello my nam
    target_txt = seq[1:] #ello my name
    return input_txt,target_txt

In [20]:
dataset = sequences.map(create_seq_targets)

In [21]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


2025-05-07 23:25:11.049711: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
batch_size = 128

In [23]:
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder = True)

In [24]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

In [25]:
vocab_size =len(vocab)

In [26]:
embed_dim = 64

In [27]:
rnn_neurons = 1026

In [28]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [29]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [30]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import  Embedding,GRU, Dense

In [65]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size, seq_len):
    model = Sequential()
    
    model.add(Input(batch_shape=(batch_size, seq_len)))  # ✅ specify fixed batch size here

    model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=seq_len))

    model.add(GRU(rnn_neurons,
                  return_sequences=True,
                  stateful=True,
                  recurrent_initializer='glorot_uniform'))
    
    model.add(Dense(vocab_size))
    
    model.compile(optimizer='adam', loss=sparse_cat_loss)
    return model


In [66]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size, seq_len)
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ (64, 100, 64)          │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ (64, 100, 128)         │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (64, 100, 10000)       │     1,290,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,004,496 (7.65 MB)

 Trainable params: 2,004,496 (7.65 MB)

 Non-trainable params: 0 (0.00 B)